Для моего проекта я использую следующие модули: bs4 [BeautifulSoup] (для скачивания данных с сайта), pymorphy2 [MorphAnalyzer] (для морфологического анализа), requests (для получения доступа к сайту), string [punctuation] для получения всех знаков препинания, re (для создания паттернов для разделения на предложения и для поиска ngram-ов).

In [ ]:
from bs4 import BeautifulSoup
from string import punctuation
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import requests
session = requests.session()
import re

C помощью функции tokenization я токенизирую предложения, очищаю их от знаков препинания и при необходимости провожу лемматизацию (что задается логической переменной flag).

In [ ]:
def tokenization(feedback, flag):
  liste = []
  for i in feedback.split():
    i = i.strip(punctuation)
    if i.isalpha():
      if flag == True:
        i = morph.parse(i.lower())[0].normal_form
      liste.append(i.lower())
  return liste

Функция pos определяет часть речи слов поступившего к ней предложения. Функция возвращает результат в виде последовательности тегов частей речи каждого из слов предложения.

In [ ]:
def pos(mot):
  for i in range(len(mot)):
    mot[i] = str(morph.parse(mot[i])[0].tag.POS)
  return ' '.join(mot)

Данная функция возвращает последовательность частеречных тегов для слов предложения, искючая слова, которые были даны в исходном запросе (при смешанном исходном запросе - слово & тег).

In [ ]:
def pos_word(word, untouched):
  for i in range(len(word)):
    if word[i] not in untouched:
      word[i] = str(morph.parse(word[i])[0].tag.POS)
  return ' '.join(word)

Скачиваем с сайта тексты произведений одного автора и их названия. Тексты разделяем на предложения с помощью регулярного выражения. В исходно созданный словарь по ключу названия произведения записываем в качестве значения список предложений текста.

In [ ]:
title_phrases = dict()
for n in range(0, 101, 50):
  link = "https://proza.ru/avtor/adacva1&s=" + str(n)
  soup = BeautifulSoup(session.get(link).text)
  refer = soup.find_all('a', {'class':"poemlink"})
  list_of_phrases = []
  for i in refer:
    story = 'https://proza.ru' + i['href']
    soup1 = BeautifulSoup(session.get(story).text)
    text = soup1.find('div', {'class':'text'})
    title = soup1.find('h1')
    title_phrases[title.text] = text.text
    pattern = re.compile(r'([А-ЯA-Z]((т.п.|т.д.|пр.|г.)|[^?!.\(]|\([^\)]*\))*[.?!])')
    for j, sent in enumerate(pattern.findall(title_phrases[title.text])):
        list_of_phrases.append(sent[0].replace("\n", " "))
    title_phrases[title.text] = list_of_phrases.copy()
    list_of_phrases.clear()

Функция, осуществляющая непосредственно поиск по заданному шаблону - search. Для начала проверяем имеются ли в заданном шаблоне двойные кавычки или одинарные, записываем это в переменную fla. Проверяем содержатся ли в запросе теги, и содержатся ли только теги или теги + слова - отмечаем это в соответствующих логических переменных. Далее проходимся по ключам & значениям словаря, конкретно по значениям. Затем исходя из типа запроса действуем определенным образом, возможные варианты: только теги, теги + слова, слова только в одной форме, слова во всех формах. Если по шаблону находится искомый запрос, то печатается предложение. В конце выдается общее число найденных предложенных.

In [ ]:
def search(sequence):
  count = 0
  parts_os = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS',
      'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ',
      'PRCL', 'INTJ']
  if sequence[0] == '"':
    fla = False
  else:
    fla = True
  pos_search = False
  mix = False
  untouched = []
  for i in sequence.split():
    if i in parts_os:
      pos_search = True
    else:
      mix = True
      untouched.append(i)
  for i in title_phrases.items():
    k = i[1]
    for j in k:
      if pos_search == True and mix == False:
        pdd = pos(tokenization(j, flag=False))
        pdd_res = re.search(r'\b' + re.escape(sequence) + r'\b', pdd)
        if pdd_res:
          print(j, "[Из произведения", '"' + i[0] + '"], proza.ru')
          count += 1
      elif pos_search == True and mix == True:
        pdd = pos_word(tokenization(j, flag=False), untouched)
        pdd_res = re.search(r'\b' + re.escape(sequence) + r'\b', pdd)
        if pdd_res:
          print(j, "[Из произведения", '"' + i[0] + '"], proza.ru')
          count += 1
      else:
        if fla == True:
          p = ' '.join(tokenization(j, flag=True))
          p2 = ' '.join(tokenization(j, flag=False))
          tok = ' '.join(tokenization(sequence.strip('"'), flag=True))
          res = re.search(r'\b' + re.escape(tok) + r'\b', p)
          res1 = re.search(r'\b' + re.escape(sequence.strip('"')) + r'\b', p2)
          if res or res1:
            print(j, "[Из произведения", '"' + i[0] + '", proza.ru]')
            count += 1
        elif fla == False:
          p2 = ' '.join(tokenization(j, flag=False))
          res1 = re.search(r'\b' + re.escape(sequence.strip('"')) + r'\b', p2)
          if res1:
            print(j, "[Из произведения", '"' + i[0] + '", proza.ru]')
            count += 1
  return 'По запросу ' + sequence + ' найдено ' + str(count) + ' результатов'

In [ ]:
print(search('ADJF жизнь'))

Военную жизнь видел. [Из произведения "Дружба - больше дружбы"], proza.ru
Наверное, на всю жизнь шум мне тогда и получила. [Из произведения "Наследство. К нотариусу"], proza.ru
Тебя, мальчик, буду помнить всю жизнь, - подумала Нина. [Из произведения "Погреб"], proza.ru
Вера Ивановна мыла посуду и отрывками слушала жалобы женщины на свою жизнь, а больше на мужа, которая с экрана без всякого стеснения вещала об этом. [Из произведения "О любви по телевизору"], proza.ru
Мне на всю жизнь хватит. [Из произведения "День с собой"], proza.ru
Как можно представить такую жизнь, если даже государство, созданное при ее жизни,  не сохранилось. [Из произведения "1. Нора Пфейффер - поэт. Судьба впереди поэзии"], proza.ru
Наступала другая жизнь, другие проблемы и другие заботы. [Из произведения "1. Нора Пфейффер - поэт. Судьба впереди поэзии"], proza.ru
Книг читать Алексей не любил, реальная жизнь давала ему больше материала для размышлений. [Из произведения "Вдвоем. В пути"], proza.ru
Так это такая жи